In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray

# Constants
WIDTH, HEIGHT = 800, 600
SPHERES = 20
FRAMES = 180
BLOCK_SIZE = (16, 16, 1)

# CUDA kernel
cuda_code = """
#include <curand_kernel.h>

struct Sphere {
    float3 center;
    float radius;
    float3 color;
    float3 velocity;
    float reflectivity;
};

__device__ float3 operator+(const float3 &a, const float3 &b) {
    return make_float3(a.x + b.x, a.y + b.y, a.z + b.z);
}

__device__ float3 operator-(const float3 &a, const float3 &b) {
    return make_float3(a.x - b.x, a.y - b.y, a.z - b.z);
}

__device__ float3 operator*(const float3 &a, float t) {
    return make_float3(a.x * t, a.y * t, a.z * t);
}

__device__ float3 operator*(float t, const float3 &a) {
    return make_float3(a.x * t, a.y * t, a.z * t);
}

__device__ float dot(const float3 &a, const float3 &b) {
    return a.x * b.x + a.y * b.y + a.z * b.z;
}

__device__ float3 normalize(const float3 &v) {
    float inv_len = rsqrtf(dot(v, v));
    return v * inv_len;
}

__device__ float3 reflect(const float3 &v, const float3 &n) {
    return v - 2.0f * dot(v, n) * n;
}

__device__ bool ray_sphere_intersect(const float3 &ray_origin, const float3 &ray_dir, const Sphere &sphere, float &t) {
    float3 oc = ray_origin - sphere.center;
    float a = dot(ray_dir, ray_dir);
    float b = 2.0f * dot(oc, ray_dir);
    float c = dot(oc, oc) - sphere.radius * sphere.radius;
    float discriminant = b * b - 4 * a * c;
    
    if (discriminant > 0) {
        float temp = (-b - sqrtf(discriminant)) / (2.0f * a);
        if (temp > 0.001f) {
            t = temp;
            return true;
        }
    }
    return false;
}

__device__ float3 ray_color(const float3 &ray_origin, const float3 &ray_dir, Sphere *spheres, int sphere_count, curandState *rand_state) {
    float3 attenuation = make_float3(1.0f, 1.0f, 1.0f);
    float3 current_origin = ray_origin;
    float3 current_dir = ray_dir;
    
    for (int bounce = 0; bounce < 3; bounce++) {
        float closest_t = INFINITY;
        int hit_sphere_index = -1;
        
        for (int i = 0; i < sphere_count; i++) {
            float t;
            if (ray_sphere_intersect(current_origin, current_dir, spheres[i], t)) {
                if (t < closest_t) {
                    closest_t = t;
                    hit_sphere_index = i;
                }
            }
        }
        
        if (hit_sphere_index == -1) {
            float3 unit_dir = normalize(current_dir);
            float t = 0.5f * (unit_dir.y + 1.0f);
            float3 sky_color = (1.0f - t) * make_float3(1.0f, 1.0f, 1.0f) + t * make_float3(0.5f, 0.7f, 1.0f);
            return attenuation * sky_color;
        }
        
        Sphere hit_sphere = spheres[hit_sphere_index];
        float3 hit_point = current_origin + closest_t * current_dir;
        float3 normal = normalize(hit_point - hit_sphere.center);
        
        float3 target = hit_point + normal + make_float3(
            curand_uniform(rand_state) - 0.5f,
            curand_uniform(rand_state) - 0.5f,
            curand_uniform(rand_state) - 0.5f
        );
        
        float3 scatter_dir = normalize(target - hit_point);
        float3 specular_dir = reflect(current_dir, normal);
        float3 final_dir = normalize(hit_sphere.reflectivity * specular_dir + (1 - hit_sphere.reflectivity) * scatter_dir);
        
        current_origin = hit_point;
        current_dir = final_dir;
        attenuation = attenuation * hit_sphere.color;
    }
    
    return make_float3(0.0f, 0.0f, 0.0f);
}

__global__ void render_kernel(uchar4 *d_output, Sphere *spheres, int sphere_count, int width, int height, curandState *rand_states) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    if (x >= width || y >= height) return;
    
    int pixel_index = y * width + x;
    curandState local_rand_state = rand_states[pixel_index];
    
    float3 ray_origin = make_float3(0.0f, 0.0f, -10.0f);
    float u = float(x) / float(width);
    float v = float(y) / float(height);
    float3 ray_dir = normalize(make_float3(u - 0.5f, v - 0.5f, 1.0f));
    
    float3 pixel_color = make_float3(0.0f, 0.0f, 0.0f);
    int samples = 4;
    
    for (int s = 0; s < samples; s++) {
        pixel_color = pixel_color + ray_color(ray_origin, ray_dir, spheres, sphere_count, &local_rand_state);
    }
    
    pixel_color = pixel_color * (1.0f / float(samples));
    pixel_color.x = sqrtf(pixel_color.x);
    pixel_color.y = sqrtf(pixel_color.y);
    pixel_color.z = sqrtf(pixel_color.z);
    
    d_output[pixel_index] = make_uchar4(
        (unsigned char)(255.99f * pixel_color.x),
        (unsigned char)(255.99f * pixel_color.y),
        (unsigned char)(255.99f * pixel_color.z),
        255
    );
}

__global__ void update_spheres_kernel(Sphere *spheres, int sphere_count, float dt) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    if (index >= sphere_count) return;
    
    spheres[index].center = spheres[index].center + spheres[index].velocity * dt;
    
    if (fabsf(spheres[index].center.x) > 5.0f) spheres[index].velocity.x *= -1;
    if (fabsf(spheres[index].center.y) > 5.0f) spheres[index].velocity.y *= -1;
    if (fabsf(spheres[index].center.z) > 5.0f) spheres[index].velocity.z *= -1;
}

__global__ void init_rand_kernel(curandState *rand_states, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    if (x >= width || y >= height) return;
    
    int pixel_index = y * width + x;
    curand_init(1984, pixel_index, 0, &rand_states[pixel_index]);
}
"""

# Compile CUDA kernel
mod = SourceModule(cuda_code)
render_kernel = mod.get_function("render_kernel")
update_spheres_kernel = mod.get_function("update_spheres_kernel")
init_rand_kernel = mod.get_function("init_rand_kernel")

# Initialize spheres
spheres = np.array([{
    'center': np.random.uniform(-5, 5, 3).astype(np.float32),
    'radius': np.random.uniform(0.5, 1.5).astype(np.float32),
    'color': np.random.uniform(0, 1, 3).astype(np.float32),
    'velocity': np.random.uniform(-0.1, 0.1, 3).astype(np.float32),
    'reflectivity': np.random.uniform(0, 0.8).astype(np.float32)
} for _ in range(SPHERES)], dtype=[
    ('center', np.float32, 3),
    ('radius', np.float32),
    ('color', np.float32, 3),
    ('velocity', np.float32, 3),
    ('reflectivity', np.float32)
])

# Allocate memory on GPU
d_spheres = cuda.mem_alloc(spheres.nbytes)
cuda.memcpy_htod(d_spheres, spheres)

d_output = cuda.mem_alloc(WIDTH * HEIGHT * 4)
d_rand_states = cuda.mem_alloc(WIDTH * HEIGHT * 8)  # 8 bytes per curandState

# Initialize random states
grid_dim = ((WIDTH + BLOCK_SIZE[0] - 1) // BLOCK_SIZE[0],
            (HEIGHT + BLOCK_SIZE[1] - 1) // BLOCK_SIZE[1])
init_rand_kernel(d_rand_states, np.int32(WIDTH), np.int32(HEIGHT),
                 block=BLOCK_SIZE, grid=grid_dim)

# Set up matplotlib for animation
fig, ax = plt.subplots(figsize=(10, 7.5))
img = ax.imshow(np.zeros((HEIGHT, WIDTH, 4), dtype=np.uint8))

def update(frame):
    # Update sphere positions
    update_spheres_kernel(d_spheres, np.int32(SPHERES), np.float32(0.1),
                          block=(256, 1, 1), grid=((SPHERES + 255) // 256, 1))
    
    # Render frame
    render_kernel(d_output, d_spheres, np.int32(SPHERES), np.int32(WIDTH), np.int32(HEIGHT),
                  d_rand_states, block=BLOCK_SIZE, grid=grid_dim)
    
    # Copy result from device to host
    h_output = np.empty((HEIGHT, WIDTH, 4), dtype=np.uint8)
    cuda.memcpy_dtoh(h_output, d_output)
    
    img.set_array(h_output)
    return [img]

# Create animation
anim = FuncAnimation(fig, update, frames=FRAMES, interval=50, blit=True)
plt.show()

# Clean up
cuda.Context.synchronize()

CompileError: nvcc preprocessing of C:\Users\sc23gd\AppData\Local\Temp\tmpb0zzosca.cu failed
[command: nvcc --preprocess -arch sm_89 -m64 -Ic:\Users\sc23gd\AppData\Local\anaconda3\envs\raytracing_env\Lib\site-packages\pycuda\cuda C:\Users\sc23gd\AppData\Local\Temp\tmpb0zzosca.cu --compiler-options -EP]

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from numba import cuda, float32, uint8
import math

# Constants
WIDTH, HEIGHT = 800, 600
SPHERES = 20
FRAMES = 180

@cuda.jit(device=True)
def dot(a, b):
    return a[0] * b[0] + a[1] * b[1] + a[2] * b[2]

@cuda.jit(device=True)
def normalize(v):
    length = math.sqrt(dot(v, v))
    return (v[0] / length, v[1] / length, v[2] / length)

@cuda.jit(device=True)
def reflect(v, n):
    d = dot(v, n) * 2
    return (v[0] - d * n[0], v[1] - d * n[1], v[2] - d * n[2])

@cuda.jit(device=True)
def ray_sphere_intersect(ray_origin, ray_dir, sphere_center, sphere_radius):
    oc = (ray_origin[0] - sphere_center[0], 
          ray_origin[1] - sphere_center[1], 
          ray_origin[2] - sphere_center[2])
    a = dot(ray_dir, ray_dir)
    b = 2.0 * dot(oc, ray_dir)
    c = dot(oc, oc) - sphere_radius * sphere_radius
    discriminant = b * b - 4 * a * c
    
    if discriminant > 0:
        t = (-b - math.sqrt(discriminant)) / (2.0 * a)
        if t > 0.001:
            return t
    return float('inf')

@cuda.jit(device=True)
def ray_color(ray_origin, ray_dir, spheres):
    attenuation = (1.0, 1.0, 1.0)
    
    for _ in range(3):  # Max 3 bounces
        t_min = float('inf')
        hit_sphere_idx = -1
        
        for i in range(SPHERES):
            t = ray_sphere_intersect(ray_origin, ray_dir, spheres[i]['center'], spheres[i]['radius'])
            if t < t_min:
                t_min = t
                hit_sphere_idx = i
        
        if hit_sphere_idx == -1:
            unit_dir = normalize(ray_dir)
            t = 0.5 * (unit_dir[1] + 1.0)
            sky_color = ((1.0 - t) + t * 0.5, (1.0 - t) + t * 0.7, (1.0 - t) + t)
            return (attenuation[0] * sky_color[0], attenuation[1] * sky_color[1], attenuation[2] * sky_color[2])
        
        hit_point = (ray_origin[0] + t_min * ray_dir[0],
                     ray_origin[1] + t_min * ray_dir[1],
                     ray_origin[2] + t_min * ray_dir[2])
        
        normal = normalize((hit_point[0] - spheres[hit_sphere_idx]['center'][0],
                            hit_point[1] - spheres[hit_sphere_idx]['center'][1],
                            hit_point[2] - spheres[hit_sphere_idx]['center'][2]))
        
        reflected = reflect(ray_dir, normal)
        ray_origin = hit_point
        ray_dir = normalize((reflected[0] + normal[0] * 0.1 * (cuda.random.xoroshiro128p_uniform_float32(rng_states, thread_id) - 0.5),
                             reflected[1] + normal[1] * 0.1 * (cuda.random.xoroshiro128p_uniform_float32(rng_states, thread_id) - 0.5),
                             reflected[2] + normal[2] * 0.1 * (cuda.random.xoroshiro128p_uniform_float32(rng_states, thread_id) - 0.5)))
        
        attenuation = (attenuation[0] * spheres[hit_sphere_idx]['color'][0],
                       attenuation[1] * spheres[hit_sphere_idx]['color'][1],
                       attenuation[2] * spheres[hit_sphere_idx]['color'][2])
    
    return (0.0, 0.0, 0.0)

@cuda.jit
def render_kernel(output, spheres, rng_states):
    x, y = cuda.grid(2)
    if x >= WIDTH or y >= HEIGHT:
        return
    
    thread_id = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    
    ray_origin = (0.0, 0.0, -10.0)
    u = float(x) / float(WIDTH)
    v = float(y) / float(HEIGHT)
    ray_dir = normalize((u - 0.5, v - 0.5, 1.0))
    
    color = (0.0, 0.0, 0.0)
    samples = 4
    
    for _ in range(samples):
        sample_color = ray_color(ray_origin, ray_dir, spheres)
        color = (color[0] + sample_color[0], color[1] + sample_color[1], color[2] + sample_color[2])
    
    color = (math.sqrt(color[0] / samples), math.sqrt(color[1] / samples), math.sqrt(color[2] / samples))
    
    idx = (y * WIDTH + x) * 3
    output[idx] = int(min(color[0], 1.0) * 255)
    output[idx + 1] = int(min(color[1], 1.0) * 255)
    output[idx + 2] = int(min(color[2], 1.0) * 255)

@cuda.jit
def update_spheres_kernel(spheres, dt):
    i = cuda.grid(1)
    if i >= SPHERES:
        return
    
    spheres[i]['center'][0] += spheres[i]['velocity'][0] * dt
    spheres[i]['center'][1] += spheres[i]['velocity'][1] * dt
    spheres[i]['center'][2] += spheres[i]['velocity'][2] * dt
    
    if abs(spheres[i]['center'][0]) > 5.0:
        spheres[i]['velocity'][0] *= -1
    if abs(spheres[i]['center'][1]) > 5.0:
        spheres[i]['velocity'][1] *= -1
    if abs(spheres[i]['center'][2]) > 5.0:
        spheres[i]['velocity'][2] *= -1

# Initialize spheres
spheres = np.array([(np.random.uniform(-5, 5, 3),  # center
                     np.random.uniform(0.5, 1.5),  # radius
                     np.random.uniform(0, 1, 3),   # color
                     np.random.uniform(-0.1, 0.1, 3))  # velocity
                    for _ in range(SPHERES)],
                   dtype=[('center', np.float32, 3),
                          ('radius', np.float32),
                          ('color', np.float32, 3),
                          ('velocity', np.float32, 3)])

d_spheres = cuda.to_device(spheres)
d_output = cuda.device_array((HEIGHT, WIDTH, 3), dtype=uint8)

rng_states = cuda.random.create_xoroshiro128p_states(WIDTH * HEIGHT, seed=1)

# Set up matplotlib for animation
fig, ax = plt.subplots(figsize=(10, 7.5))
img = ax.imshow(np.zeros((HEIGHT, WIDTH, 3), dtype=np.uint8))

def update(frame):
    update_spheres_kernel[SPHERES, 1](d_spheres, 0.1)
    render_kernel[(WIDTH + 15) // 16, (HEIGHT + 15) // 16, 1](d_output, d_spheres, rng_states)
    img.set_array(d_output.copy_to_host())
    return [img]

# Create animation
anim = FuncAnimation(fig, update, frames=FRAMES, interval=50, blit=True)
plt.show()

# Clean up
cuda.synchronize()

RuntimeError: Numba cannot operate on non-primary CUDA context 24bb37903a0